In [ ]:
!cd /content/drive/MyDrive
!mkdir -p NN
!cd NN


In [4]:
!git clone https://github.com/FatemaKotb/An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image.git
!cd An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image


Cloning into 'An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 190 (delta 77), reused 151 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (190/190), 1.81 MiB | 12.96 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [2]:
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [3]:
!pip install timm opencv-python scipy matplotlib tqdm


In [19]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


False


AssertionError: Torch not compiled with CUDA enabled

In [20]:
# Correct way to change directory in Colab
%cd /content/An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image

# Confirm
!pwd
!ls


/content/An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image
/content/An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image
clip_finetune  food_vision_playground  README.md


In [21]:
!ls

clip_finetune  food_vision_playground  README.md


In [22]:
!git config user.name "maryammmabdelrahman-eng"
!git config user.email "maryammmabdelrahman@gmail.com"

In [23]:
!git checkout -b maskedrcnn_clip_pipeline


fatal: A branch named 'maskedrcnn_clip_pipeline' already exists.


In [24]:
# 1️⃣ Install dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu118/torch2.1/index.html
!pip install opencv-python scipy tqdm

Looking in indexes: https://download.pytorch.org/whl/cu118
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu118/torch2.1/index.html
ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [15]:

# ===============================
# Create dataset directory in Drive
# ===============================
DATASET_DIR = "/content/drive/MyDrive/datasets/FoodSeg103"
!mkdir -p "$DATASET_DIR"

# ===============================
# Download FoodSeg103-256 zip into Drive
# ===============================
!wget https://zenodo.org/record/15729578/files/FoodSeg103-256.zip \
    -O "$DATASET_DIR/FoodSeg103-256.zip"

# ===============================
# Unzip inside Drive
# ===============================
!unzip -qq "$DATASET_DIR/FoodSeg103-256.zip" -d "$DATASET_DIR"

# ===============================
# Verify contents
# ===============================
!ls "$DATASET_DIR"

--2025-12-28 14:03:12--  https://zenodo.org/record/15729578/files/FoodSeg103-256.zip
Resolving zenodo.org (zenodo.org)... 188.185.43.153, 188.185.48.75, 137.138.52.235, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.153|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/15729578/files/FoodSeg103-256.zip [following]
--2025-12-28 14:03:14--  https://zenodo.org/records/15729578/files/FoodSeg103-256.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1199113322 (1.1G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/datasets/FoodSeg103/FoodSeg103-256.zip’

/content/drive/MyDr 100%[===================>]   1.12G   821KB/s    in 26m 24s 

2025-12-28 14:29:38 (739 KB/s) - ‘/content/drive/MyDrive/datasets/FoodSeg103/FoodSeg103-256.zip’ saved [1199113322/1199113322]

FoodSeg103-256	FoodSeg103-256.zip


In [ ]:
!ls

datasets  food_vision_playground  README.md


In [ ]:
# Install tree if needed
!apt-get install -qq tree


Selecting previously unselected package tree.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# Show full tree of dataset
!tree -L 3 /content/An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image/datasets/FoodSeg103/FoodSeg103-256


/content/An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image/datasets/FoodSeg103/FoodSeg103-256
├── category_id.txt
├── Images
│   ├── ann_dir
│   │   ├── test
│   │   └── train
│   └── img_dir
│       ├── test
│       └── train
├── ImageSets
│   ├── test.txt
│   └── train.txt
├── InstanceMasks
├── Readme.txt
├── test_recipe1m_id.txt
└── train_test_recipe1m_id.txt

9 directories, 6 files


In [25]:
# Imports
import os
from PIL import Image
import matplotlib.pyplot as plt

# Base dataset path
DATA_DIR = DATASET_DIR
# Image and annotation directories
IMG_DIR_TRAIN = os.path.join(DATA_DIR, "Images", "img_dir", "train")
IMG_DIR_TEST = os.path.join(DATA_DIR, "Images", "img_dir", "test")

MASK_DIR_TRAIN = os.path.join(DATA_DIR, "Images", "ann_dir", "train")
MASK_DIR_TEST = os.path.join(DATA_DIR, "Images", "ann_dir", "test")

# Read train/test file lists
train_files = [x.strip() for x in open(os.path.join(DATA_DIR, "ImageSets", "train.txt")).readlines()]
test_files = [x.strip() for x in open(os.path.join(DATA_DIR, "ImageSets", "test.txt")).readlines()]

# Strip .jpg from IDs
train_files = [f.replace(".jpg", "") for f in train_files]
test_files = [f.replace(".jpg", "") for f in test_files]

# Sanity check
print("Sample IDs from train.txt:", train_files[:5])

# Function to get full path of image/mask given a file ID
def get_image_mask_paths(file_id, train=True):
    img_dir = IMG_DIR_TRAIN if train else IMG_DIR_TEST
    mask_dir = MASK_DIR_TRAIN if train else MASK_DIR_TEST
    img_path = os.path.join(img_dir, f"{file_id}.bmp")
    mask_path = os.path.join(mask_dir, f"{file_id}.bmp")
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Image not found: {img_path}")
    if not os.path.exists(mask_path):
        raise FileNotFoundError(f"Mask not found: {mask_path}")
    return img_path, mask_path

# Load a sample image/mask
sample_id = train_files[0]  # e.g., "00000000"
sample_img_path, sample_mask_path = get_image_mask_paths(sample_id, train=True)

sample_img = Image.open(sample_img_path).convert("RGB")
sample_mask = Image.open(sample_mask_path)

# Display sample image and mask
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.title("Image")
plt.imshow(sample_img)
plt.axis("off")

plt.subplot(1,2,2)
plt.title("Mask")
plt.imshow(sample_mask)
plt.axis("off")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/datasets/FoodSeg103/ImageSets/train.txt'

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import functional as F
import numpy as np

class FoodSegDataset(Dataset):
    def __init__(self, img_ids, img_dir, mask_dir, transforms=None):
        self.img_ids = img_ids
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        file_id = self.img_ids[idx]
        img_path = os.path.join(self.img_dir, f"{file_id}.bmp")
        mask_path = os.path.join(self.mask_dir, f"{file_id}.bmp")

        # Load image and mask
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)

        # Convert mask to numpy array
        mask = np.array(mask)

        # If the mask is multi-class, convert each instance into a separate binary mask
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[obj_ids != 0]  # remove background

        masks = mask == obj_ids[:, None, None]  # shape: [num_objs, H, W]

        # Bounding boxes [x_min, y_min, x_max, y_max]
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # Convert to torch tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        labels = torch.ones((num_objs,), dtype=torch.int64)  # all 1s (food class)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "masks": masks,
            "image_id": image_id,
            "area": area,
            "iscrowd": iscrowd
        }

        if self.transforms:
            img, target = self.transforms(img, target)

        img = F.to_tensor(img)

        return img, target

# Create train and test datasets
train_dataset = FoodSegDataset(
    img_ids=train_files,
    img_dir=IMG_DIR_TRAIN,
    mask_dir=MASK_DIR_TRAIN
)

test_dataset = FoodSegDataset(
    img_ids=test_files,
    img_dir=IMG_DIR_TEST,
    mask_dir=MASK_DIR_TEST
)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Test a batch
images, targets = next(iter(train_loader))
print("Batch images:", len(images))
print("Batch targets:", targets[0])


NameError: name 'train_files' is not defined

In [ ]:
# ----------------------------
# Imports
# ----------------------------
import os
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# ----------------------------
# Dataset Setup
# ----------------------------
DATA_DIR = "/content/drive/MyDrive/datasets/FoodSeg103/FoodSeg103-256"

IMG_DIR_TRAIN = os.path.join(DATA_DIR, "Images", "img_dir", "train")
IMG_DIR_TEST = os.path.join(DATA_DIR, "Images", "img_dir", "test")
MASK_DIR_TRAIN = os.path.join(DATA_DIR, "Images", "ann_dir", "train")
MASK_DIR_TEST = os.path.join(DATA_DIR, "Images", "ann_dir", "test")

train_files = [x.strip().replace(".jpg", "") for x in open(os.path.join(DATA_DIR, "ImageSets", "train.txt")).readlines()]
test_files = [x.strip().replace(".jpg", "") for x in open(os.path.join(DATA_DIR, "ImageSets", "test.txt")).readlines()]

print("Number of train images:", len(train_files))
print("Number of test images:", len(test_files))

# ----------------------------
# Dataset Class
# ----------------------------
class FoodSegDataset(Dataset):
    def __init__(self, file_ids, img_dir, mask_dir, transforms=None):
        self.file_ids = file_ids
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.file_ids)

    def __getitem__(self, idx):
        file_id = self.file_ids[idx]
        img_path = os.path.join(self.img_dir, f"{file_id}.bmp")
        mask_path = os.path.join(self.mask_dir, f"{file_id}.bmp")

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
        mask = np.array(mask)

        # Convert mask to instance labels (1..N)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[obj_ids != 0]  # remove background
        masks = mask == obj_ids[:, None, None]

        boxes = []
        for m in masks:
            pos = np.where(m)
            xmin = int(np.min(pos[1]))
            xmax = int(np.max(pos[1]))
            ymin = int(np.min(pos[0]))
            ymax = int(np.max(pos[0]))
            # Ensure box is valid
            if xmax > xmin and ymax > ymin:
                boxes.append([xmin, ymin, xmax, ymax])
            else:
                boxes.append([0, 0, 1, 1])  # fallback

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((len(boxes),), dtype=torch.int64)  # one label per instance
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        target = {"boxes": boxes, "labels": labels, "masks": masks}

        if self.transforms:
            img = self.transforms(img)
        else:
            img = F.to_tensor(img)

        return img, target

# ----------------------------
# DataLoaders
# ----------------------------
train_dataset = FoodSegDataset(train_files, IMG_DIR_TRAIN, MASK_DIR_TRAIN)
test_dataset = FoodSegDataset(test_files, IMG_DIR_TEST, MASK_DIR_TEST)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# ----------------------------
# Model
# ----------------------------
num_classes = 104  # 103 foods + background
model = maskrcnn_resnet50_fpn(weights="DEFAULT")  # pretrained on COCO

# Replace the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Replace the mask predictor
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# ----------------------------
# Optimizer & Scheduler
# ----------------------------
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# ----------------------------
# Training Loop
# ----------------------------
num_epochs = 20
train_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for imgs, targets in train_loader:
        imgs = list(img.to(device) for img in imgs)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

    lr_scheduler.step()
    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    # Visualize after every 5 epochs
    if (epoch+1) % 5 == 0:
        model.eval()
        sample_img, _ = train_dataset[0]
        with torch.no_grad():
            prediction = model([sample_img.to(device)])
        masks = prediction[0]['masks'].cpu().numpy()
        plt.figure(figsize=(10,5))
        plt.imshow(sample_img.permute(1,2,0))
        for i in range(min(3, masks.shape[0])):  # show first 3 masks
            plt.imshow(masks[i,0], alpha=0.5)
        plt.title(f"Epoch {epoch+1} Sample Prediction")
        plt.axis("off")
        plt.show()

# ----------------------------
# Loss Curve
# ----------------------------
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses, marker='o')
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

# ----------------------------
# Testing (sample)
# ----------------------------
model.eval()
with torch.no_grad():
    sample_img, _ = test_dataset[0]
    prediction = model([sample_img.to(device)])
    masks = prediction[0]['masks'].cpu().numpy()

plt.figure(figsize=(10,5))
plt.imshow(sample_img.permute(1,2,0))
for i in range(min(3, masks.shape[0])):
    plt.imshow(masks[i,0], alpha=0.5)
plt.title("Sample Test Prediction")
plt.axis("off")
plt.show()


Number of train images: 4983
Number of test images: 2135
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


100%|██████████| 170M/170M [00:02<00:00, 75.2MB/s]


In [ ]:
# ----------------------------
# Colab Script: Integrate RCNN into GitHub Pipeline
# ----------------------------

import os
import torch
from pathlib import Path
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from scripts.factory import PipelineFactoryConfig, build_default_pipeline
from scripts.run_pipeline import load_rgb, ensure_uint8_rgb, _setup_logger

# ----------------------------
# Paths
# ----------------------------
REPO_DIR = "/content/An-Instance-Aware-Multi-Task-Network-for-Food-Calorie-Estimation-from-a-Single-Image"
MODEL_PATH = os.path.join(REPO_DIR, "models", "rcnn_foodseg103.pth")
TEST_IMAGE = os.path.join(REPO_DIR, "datasets/FoodSeg103/FoodSeg103-256/Images/img_dir/test/00000000.bmp")

os.makedirs(os.path.join(REPO_DIR, "models"), exist_ok=True)

# ----------------------------
# 1) Save trained RCNN (already in memory as `model`)
# ----------------------------
torch.save(model.state_dict(), MODEL_PATH)
print(f"Saved trained RCNN to {MODEL_PATH}")

# ----------------------------
# 2) Edit factory.py to support custom RCNN
# ----------------------------
FACTORY_FILE = os.path.join(REPO_DIR, "scripts/factory.py")
with open(FACTORY_FILE, "r") as f:
    factory_code = f.read()

if "custom_seg_model" not in factory_code:
    factory_code = factory_code.replace(
        "def build_default_pipeline(cfg: PipelineFactoryConfig):",
        "def build_default_pipeline(cfg: PipelineFactoryConfig, custom_seg_model=None):"
    ).replace(
        "seg_block = SegmentationBlock(cfg.device)",
        "seg_block = custom_seg_model if custom_seg_model is not None else SegmentationBlock(cfg.device)"
    )

with open(FACTORY_FILE, "w") as f:
    f.write(factory_code)
print("Edited factory.py to support custom RCNN")

# ----------------------------
# 3) Load RCNN and build pipeline
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

num_classes = 104
rcnn_model = maskrcnn_resnet50_fpn(weights=None)
in_features_box = rcnn_model.roi_heads.box_predictor.cls_score.in_features
rcnn_model.roi_heads.box_predictor = torch.nn.Linear(in_features_box, num_classes)

in_features_mask = rcnn_model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
rcnn_model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

rcnn_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
rcnn_model.to(device).eval()
print("Loaded RCNN model for pipeline")

# ----------------------------
# 4) Build pipeline using trained RCNN
# ----------------------------
cfg = PipelineFactoryConfig(device=device, seg_score_thresh=0.5)
pipe = build_default_pipeline(cfg, custom_seg_model=rcnn_model)
print("Pipeline built with RCNN as segmentation block")

# ----------------------------
# 5) Run a test image
# ----------------------------
logger = _setup_logger()
img = ensure_uint8_rgb(load_rgb(TEST_IMAGE))
out = pipe(img)
print(f"Detected {len(out.instance_outputs)} instances in test image {TEST_IMAGE}")

# Display masks overlay
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
plt.imshow(img)
for i, inst in enumerate(out.instance_outputs[:5]):
    plt.imshow(inst.mask, alpha=0.5)
plt.title("Top-5 RCNN Segmentation Masks Overlay")
plt.axis("off")
plt.show()
